In [1]:
import os
import re
import subprocess
from pathlib import Path

from wombat.utils import listfiles
from wombat.bsub import batch_bsub_commands, DEFAULT_ARGS, write_command_file

In [2]:
# !pip install --force-reinstall git+https://github.com/estorrs/wombat

In [3]:
# !pip install git+https://github.com/estorrs/wombat.git@2debd536de6a42d2f550366f27e708d2fa9cd09e

python main.py -K=20 --dataset="path/to/simulation 1" --repli_list="[1,3]" --use_spatial="[True]*2" --result_filename="SpiceMix_K20_FOV13.h5"

In [ ]:
# in_dir = '/storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/analysis/sc_pdac/HT242P1-H1/inputs'
# out_dir = '/storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/analysis/sc_pdac/HT242P1-H1'

In [4]:
in_dir = '/storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/analysis/brca_dcis/combined_run_1/inputs'
out_dir = '/storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/analysis/brca_dcis/combined_run_1'

In [5]:
K = 20
out = os.path.join(out_dir, 'output.h5')

log_dir = os.path.join(out_dir, 'logs')
Path(log_dir).mkdir(parents=True, exist_ok=True)

In [6]:
fovs = [re.sub(r'^.*expression_(.*).txt$', r'\1', fp) for fp in sorted(listfiles(in_dir, regex='expression'))]
bools = ['True'] * len(fovs)

fov_str = '[' + ','.join(fovs) + ']'
bool_str = '[' + ','.join(bools) + ']'

In [7]:
cwd = '/storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/SpiceMix/'
command = f'cd {cwd} && python main.py -K={K} --path2dataset={in_dir} --repli_list="{fov_str}" --use_spatial="{bool_str}" --result_filename="{out}" --expression_suffix="" --neighbor_suffix=""'

command


'cd /storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/SpiceMix/ && python main.py -K=20 --path2dataset=/storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/analysis/brca_dcis/combined_run_1/inputs --repli_list="[0,1,10,11,12,13,14,15,16,17,2,3,4,5,6,7,8,9]" --use_spatial="[True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True]" --result_filename="/storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/analysis/brca_dcis/combined_run_1/output.h5" --expression_suffix="" --neighbor_suffix=""'

In [8]:
args = DEFAULT_ARGS
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': None,
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [9]:
args['mem'] = 100
args['docker'] = 'estorrs/spicemix:75522de'
args['queue'] = 'general'
args['use_gpu'] = True
exports = ['/miniconda/envs/SpiceMix/bin']

In [10]:
cmds = batch_bsub_commands([command], ['spicemix_run_1'], log_dir, args, exports=exports)
cmds

['mkdir -p /storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/analysis/brca_dcis/combined_run_1/logs',
 'export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab:/storage1/fs1/dinglab /scratch1/fs1/dinglab:/scratch1/fs1/dinglab /home/estorrs:/home/estorrs /storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/analysis/brca_dcis/combined_run_1/logs:/storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/analysis/brca_dcis/combined_run_1/logs"',
 'export PATH="/opt/java/openjdk/bin:/miniconda/envs/SpiceMix/bin:$PATH"',
 'bsub -R \'select[gpuhost,mem>100GB] rusage[mem=100GB] span[hosts=1]\' -M 101GB -gpu \'num=1:gmodel=TeslaV100_SXM2_32GB:gmem=30GB\' -n 1 -q general -G compute-dinglab -a \'docker(estorrs/spicemix:75522de)\' -oo /storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/analysis/brca_dcis/combined_run_1/logs/spicemix_run_1.txt \'cd /storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/SpiceMix/ && python main.py -K=20 --path2dataset=/storage1/fs1/dinglab/Active/Projects/estorrs/Sp

In [11]:
write_command_file(cmds, os.path.join(out_dir, 'start_spicemix.sh'))

In [5]:
# old_cwd = os.getcwd()
# os.chdir('/storage1/fs1/dinglab/Active/Projects/estorrs/SpiceMix/SpiceMix')
# subprocess.check_output(command, shell=True)